In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.models import load_model
from keras.layers import LSTM, Dense, Dropout

2024-02-05 00:49:17.379785: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
filename = 'euro-millions' 
filepath = '/Users/dantashashou/Downloads/euromillions_202002.csv'
df = pd.read_csv(filepath, encoding='ISO-8859-1', sep=';')
df.head(10)

,annee_numero_de_tirage,jour_de_tirage,date_de_tirage,numéro_de_tirage_dans_le_cycle,date_de_forclusion,boule_1,boule_2,boule_3,boule_4,boule_5,...,rapport_du_rang7_Etoile+,nombre_de_gagnant_au_rang8_Etoile+,rapport_du_rang8_Etoile+,nombre_de_gagnant_au_rang9_Etoile+,rapport_du_rang9_Etoile+,nombre_de_gagnant_au_rang10_Etoile+,rapport_du_rang10_Etoile+,numero_My_Million,numero_Tirage_Exceptionnel_Euro_Million,Unnamed: 75
0,20244010,VENDREDI,02/02/2024,1,03/04/2024,13,42,27,23,20,...,"2,9",24894,"9,3",63680,2,457429,"2,5",VE 765 3773,NaN,NaN
1,20244009,MARDI,30/01/2024,3,31/03/2024,10,27,5,30,19,...,"3,8",24563,12,89382,"1,8",581723,"2,5",SR 540 3822,NaN,NaN
2,20244008,VENDREDI,26/01/2024,Super Jackpot et tirages suivants,27/03/2024,32,19,41,8,42,...,"4,2",41128,"11,1",94649,"2,7",889755,"2,5",RU 715 8901,NaN,NaN
3,20244007,MARDI,23/01/2024,1,24/03/2024,48,23,14,50,39,...,"4,4",12899,"11,7",30850,"2,7",308046,"2,4",KR 537 5327,NaN,NaN
4,20244006,VENDREDI,19/01/2024,1,20/03/2024,50,27,28,48,44,...,4,18046,"11,1",42691,"2,6",424081,"2,3",QE 911 6397,NaN,NaN
5,20244005,MARDI,16/01/2024,7,17/03/2024,18,21,33,45,10,...,"4,4",13958,"12,7",39974,"2,5",328733,"2,7",CU 208 5890,NaN,NaN
6,20244004,VENDREDI,12/01/2024,6,13/03/2024,18,16,17,49,45,...,"4,3",18344,"12,4",46653,"2,7",427062,"2,6",RT 208 4085,NaN,NaN
7,20244003,MARDI,09/01/2024,5,10/03/2024,12,9,2,39,40,...,"3,8",15703,"10,8",41373,"2,3",339112,"2,4",LN 958 0675,NaN,NaN
8,20244002,VENDREDI,05/01/2024,4,06/03/2024,18,4,39,7,50,...,"3,1",26334,"8,5",54128,"2,3",474691,"2,3",SA 454 2143,NaN,NaN
9,20244001,MARDI,02/01/2024,3,03/03/2024,7,18,49,46,15,...,"4,7",11950,"13,3",32149,"2,8",272968,"2,9",VM 156 4461,NaN,NaN


In [3]:
print(df.columns)

Index(['annee_numero_de_tirage', 'jour_de_tirage', 'date_de_tirage',
       'numéro_de_tirage_dans_le_cycle', 'date_de_forclusion', 'boule_1',
       'boule_2', 'boule_3', 'boule_4', 'boule_5', 'etoile_1', 'etoile_2',
       'boules_gagnantes_en_ordre_croissant',
       'etoiles_gagnantes_en_ordre_croissant',
       'nombre_de_gagnant_au_rang1_Euro_Millions_en_france',
       'nombre_de_gagnant_au_rang1_Euro_Millions_en_europe',
       'rapport_du_rang1_Euro_Millions',
       'nombre_de_gagnant_au_rang2_Euro_Millions_en_france',
       'nombre_de_gagnant_au_rang2_Euro_Millions_en_europe',
       'rapport_du_rang2_Euro_Millions',
       'nombre_de_gagnant_au_rang3_Euro_Millions_en_france',
       'nombre_de_gagnant_au_rang3_Euro_Millions_en_europe',
       'rapport_du_rang3_Euro_Millions',
       'nombre_de_gagnant_au_rang4_Euro_Millions_en_france',
       'nombre_de_gagnant_au_rang4_Euro_Millions_en_europe',
       'rapport_du_rang4_Euro_Millions',
       'nombre_de_gagnant_au_rang5_Eu

In [4]:
first_row = df.iloc[0]
print(first_row)

annee_numero_de_tirage                        20244010
jour_de_tirage                                VENDREDI
date_de_tirage                              02/02/2024
numéro_de_tirage_dans_le_cycle                       1
date_de_forclusion                          03/04/2024
                                              ...     
nombre_de_gagnant_au_rang10_Etoile+             457429
rapport_du_rang10_Etoile+                          2,5
numero_My_Million                          VE 765 3773
numero_Tirage_Exceptionnel_Euro_Million            NaN
Unnamed: 75                                        NaN
Name: 0, Length: 76, dtype: object


In [5]:
df = df.iloc[:, 5:12] 
df.head()

,boule_1,boule_2,boule_3,boule_4,boule_5,etoile_1,etoile_2
0,13,42,27,23,20,9,5
1,10,27,5,30,19,6,5
2,32,19,41,8,42,12,9
3,48,23,14,50,39,12,3
4,50,27,28,48,44,7,12


In [6]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)
transformed_df.head()

,0,1,2,3,4,5,6
0,-0.929431,1.254085,0.073931,-0.208789,-0.441134,0.642982,-0.379975
1,-1.143521,0.184710,-1.488990,0.287892,-0.510015,-0.224007,-0.379975
2,0.426473,-0.385623,1.068518,-1.273106,1.074245,1.509971,0.775150
3,1.568287,-0.100456,-0.849613,1.706980,0.867602,1.509971,-0.957538
4,1.711014,0.184710,0.144973,1.565071,1.212006,0.064990,1.641493


In [7]:
number_of_rows= df.values.shape[0] #all our games
window_length = 5 #amount of past games we need to take in consideration for prediction
number_of_features = df.values.shape[1] #balls count

In [8]:
train = np.empty([number_of_rows-window_length, window_length, number_of_features], dtype=float)
label = np.empty([number_of_rows-window_length, number_of_features], dtype=float)

for i in range(0, number_of_rows-window_length):
    train[i]=transformed_df.iloc[i:i+window_length, 0: number_of_features]
    label[i]=transformed_df.iloc[i+window_length: i+window_length+1, 0: number_of_features]

In [9]:
train.shape

(413, 5, 7)

In [10]:
label.shape

(413, 7)

In [11]:
train[0]

array([[-0.92943115,  1.2540846 ,  0.07393115, -0.20878932, -0.44113409,
         0.64298196, -0.37997527],
       [-1.14352128,  0.18471013, -1.48899032,  0.28789162, -0.51001494,
        -0.22400662, -0.37997527],
       [ 0.426473  , -0.38562291,  1.06851753, -1.27310563,  1.07424472,
         1.50997053,  0.77514956],
       [ 1.56828703, -0.10045639, -0.84961335,  1.70698003,  0.86760216,
         1.50997053, -0.95753769],
       [ 1.71101378,  0.18471013,  0.14497303,  1.56507119,  1.21200643,
         0.06498957,  1.64149318]])

In [12]:
train[1]

array([[-1.14352128,  0.18471013, -1.48899032,  0.28789162, -0.51001494,
        -0.22400662, -0.37997527],
       [ 0.426473  , -0.38562291,  1.06851753, -1.27310563,  1.07424472,
         1.50997053,  0.77514956],
       [ 1.56828703, -0.10045639, -0.84961335,  1.70698003,  0.86760216,
         1.50997053, -0.95753769],
       [ 1.71101378,  0.18471013,  0.14497303,  1.56507119,  1.21200643,
         0.06498957,  1.64149318],
       [-0.57261427, -0.24303965,  0.50018246,  1.35220792, -1.12994264,
         1.50997053,  0.48636835]])

In [13]:
label[0]

array([-0.57261427, -0.24303965,  0.50018246,  1.35220792, -1.12994264,
        1.50997053,  0.48636835])

In [14]:
label[1]

array([-0.57261427, -0.5994978 , -0.6364877 ,  1.63602561,  1.28088729,
        0.64298196,  1.64149318])

In [15]:
batch_size = 25 
if os.path.exists('../input/euromillionsireland/'+filename+'.h5'):
    model = load_model('../input/euromillionsireland/'+filename+'.h5')
else:
    model = Sequential()
    model.add(LSTM(32,      
               input_shape=(window_length, number_of_features),
               return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(32,           
               return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(number_of_features))
    model.compile(loss='mse', optimizer='rmsprop')
    model.fit(train, label, batch_size=64, epochs=2000)
    # model.save('input/'+filename+'.h5')

Epoch 1/2000
7/7 [==============================] - 4s 11ms/step - loss: 1.0053
Epoch 2/2000
7/7 [==============================] - 0s 10ms/step - loss: 0.9986
Epoch 3/2000
7/7 [==============================] - 0s 11ms/step - loss: 1.0004
Epoch 4/2000
7/7 [==============================] - 0s 10ms/step - loss: 0.9985
Epoch 5/2000
7/7 [==============================] - 0s 11ms/step - loss: 0.9963
Epoch 6/2000
7/7 [==============================] - 0s 9ms/step - loss: 0.9921
Epoch 7/2000
7/7 [==============================] - 0s 11ms/step - loss: 0.9929
Epoch 8/2000
7/7 [==============================] - 0s 8ms/step - loss: 0.9910
Epoch 9/2000
7/7 [==============================] - 0s 9ms/step - loss: 0.9903
Epoch 10/2000
7/7 [==============================] - 0s 7ms/step - loss: 0.9892
Epoch 11/2000
7/7 [==============================] - 0s 7ms/step - loss: 0.9863
Epoch 12/2000
7/7 [==============================] - 0s 6ms/step - loss: 0.9862
Epoch 13/2000
7/7 [========================

In [16]:
to_predict=df.iloc[-5:]
scaled_to_predict = scaler.transform(to_predict)

In [17]:
scaled_predicted_output_1 = model.predict(np.array([scaled_to_predict]))
data = scaler.inverse_transform(scaled_predicted_output_1).astype(int)
predict = pd.DataFrame(data, columns=['boule_1', 'boule_2', 'boule_3', 'boule_4', 'boule_5', 'etoile_1', 'etoile_2'])
# df.to_csv(''+filename+'.csv', index=False)  
predict

1/1 [==============================] - 1s 1s/step


,boule_1,boule_2,boule_3,boule_4,boule_5,etoile_1,etoile_2
0,0,21,32,23,30,5,4
